In [18]:
from ripe.atlas.cousteau import Measurement, Probe, Dns, AtlasSource, AtlasCreateRequest
from datetime import datetime

In [28]:
ATLAS_API_KEY = "f00d3214-a18e-4f6d-8188-104dd1192b36"
targets = ['www.google.com', 'www.twitter.com']
blocked_list = ['xxx.xxx']
_tags = {'include': ['system-ipv4-works']}
probe = AtlasSource(type='probes', value='15397', requested=1, tags=_tags)
req_ids = []

while targets and len(targets) != 0:
    for _domain in targets:
        if _domain not in blocked_list:
            msm = Dns(af=4, query_class='IN', query_argument=_domain, query_type='A', use_probe_resolver=True,
                               include_abuf=True, retry=5, description='New DNS A request for ' + _domain)
            atlas_request = AtlasCreateRequest(
                start_time=datetime.utcnow(),
                key=ATLAS_API_KEY,
                is_oneoff=True,
                measurements=[msm],
                sources=[probe],
            )
            (is_success, response) = atlas_request.create()
            if is_success:
                req_ids += response['measurements']
                targets.remove(_domain)
            else:
                print(str(response))

In [152]:
f = open('../fresh_dns/dns_data', 'rb')
pcl = pickle.Unpickler(f)
dns_data = pcl.load()
# dns_data is dict of domain:[list]
res = dns_data['facebook'] # List of all dictionaries (subdomain:dict)

In [155]:
rr = res[4]['ar-ar.facebook.com.'] # dict of {ProbeId: DNS Result}

In [161]:
rr[15397].responses[0].abuf.answers[0].raw_data

Measurement #8334367, Probe #15397

In [139]:
f = open('../fresh_dns/dns_request_ids', 'r')
line = f.readline()
req_ids = [int(v) for v in list(line[1:-1].split(','))]
len(req_ids)

362

In [134]:
done = []
for r in req_ids:
    m = Measurement(id=r)
    done.append(m.meta_data['query_argument'])

In [136]:
f1 = open('../resources/subdomains.txt', 'r')
lines = [(line.strip() + '.') for line in f1.readlines()]
ans = set(lines).difference(set(done))

In [141]:
len(ans)

30

In [142]:
ans

{'absa.co.za.',
 'annualreport2015.standardbank.com.',
 'blog.absa.co.za.',
 'careers.standardbank.com.',
 'cib.absa.co.za.',
 'community.standardbank.co.za.',
 'corporateandinvestment.standardbank.com.',
 'gaming.supersport.com.',
 'ib.absa.co.za.',
 'images.supersport.com.',
 'instantbusiness.absa.co.za.',
 'lifeinsurance.standardbank.co.za.',
 'merchantsolutions.standardbank.co.za.',
 'reporting.standardbank.com.',
 'sponsorships.standardbank.com.',
 'standardbank.co.za.',
 'supersport.com.',
 'ucount.standardbank.co.za.',
 'www.absa.co.za.',
 'www.standardbank.cd.',
 'www.standardbank.co.ao.',
 'www.standardbank.co.mw.',
 'www.standardbank.co.mz.',
 'www.standardbank.co.sz.',
 'www.standardbank.co.za.',
 'www.standardbank.com.',
 'www.standardbank.com.na.',
 'www.standardbank.mu.',
 'www.supersport.com.',
 'www.wealthandinvestment.standardbank.com.'}

In [ ]:
final_dict = dict()
count = 1
for (domain_name, value) in ping_dict.items():
    for item in value:
        for (subdomain_name, p_list) in item.items():
            print(subdomain_name)
            for p_id in p_list.keys():
                p = Probe(id=p_id)
                c_code = p.country_code
                if c_code not in final_dict:
                    final_dict[c_code] = dict()
                if subdomain_name not in final_dict[c_code]:
                    final_dict[c_code][subdomain_name] = []
                final_dict[c_code][subdomain_name].append(p_id)
    count += 1
    if count == 2:
        exit()